In [10]:
from bitwuzla import *

In [11]:
# First, create a term manager instance.
tm = TermManager()
# Create a Bitwuzla options instance.
options = Options()
# Then, enable model generation.
options.set(Option.PRODUCE_MODELS, True)
# Create a Bitwuzla instance.
bitwuzla = Bitwuzla(tm, options)

In [12]:
# create the ys by the sbox
sbox = [4, 7, 9, 11, 12, 6, 14, 15, 0, 5, 1, 13, 8, 3, 2, 10]
sbox_inverse = [8, 10, 14, 13, 0, 9, 5, 1, 12, 2, 15, 3, 4, 11, 6, 7]
sbox_midori = [
    0xC,
    0xA,
    0xD,
    0x3,
    0xE,
    0xB,
    0xF,
    0x7,
    0x8,
    0x9,
    0x1,
    0x5,
    0x0,
    0x2,
    0x4,
    0x6,
]


def sbox_to_bits(sbox: list):

    ys = [0] * 4
    for i in range(len(sbox)):
        ys[3] = ys[3] | ((sbox[i] & 1) << (len(sbox) - 1 - i))
        ys[2] = ys[2] | (((sbox[i] >> 1) & 1) << (len(sbox) - 1 - i))
        ys[1] = ys[1] | (((sbox[i] >> 2) & 1) << (len(sbox) - 1 - i))
        ys[0] = ys[0] | (((sbox[i] >> 3) & 1) << (len(sbox) - 1 - i))
    return ys


ys = sbox_to_bits(sbox)
for y in range(4):
    print(hex(ys[y]))
ys_inv = sbox_to_bits(sbox_inverse)
for y in range(4):
    print(hex(ys_inv[y]))
ys = sbox_to_bits(sbox_midori)
for y in range(4):
    print(hex(ys[y]))

0x3b19
0xcf50
0x5707
0x7174
0xf4a4
0x32ab
0x6077
0x1735
0xeec0
0xab13
0x5f05
0x3770


In [13]:
bits = 4
bit_with = 2**bits
gate_number = 20
# XOR, AND, OR, NOT
gts = [0b0110, 0b001, 0b0111, 0b1100]
# this is the sbox, it is temp used, after need generate by the true sbox
xs = [0x00FF, 0x0F0F, 0x3333, 0x5555]
ys = [0xEEC0, 0xAB13, 0x5F05, 0x3770]

sortbv = tm.mk_bv_sort(bit_with)

In [14]:
# create the input and output variables
xs_v = [tm.mk_const(sortbv, "x{}".format(i)) for i in range(bits)]
ys_v = [tm.mk_const(sortbv, "y{}".format(i)) for i in range(bits)]

# create the constraints
for i in range(bits):
    bitwuzla.assert_formula(
        tm.mk_term(Kind.EQUAL, [xs_v[i], tm.mk_bv_value(sortbv, xs[i])])
    )
for i in range(bits):
    bitwuzla.assert_formula(
        tm.mk_term(Kind.EQUAL, [ys_v[i], tm.mk_bv_value(sortbv, ys[i])])
    )

In [15]:
# create the gate input and output
ts_v = [tm.mk_const(sortbv, "t{}".format(i)) for i in range(gate_number)]
qs_v = [tm.mk_const(sortbv, "q{}".format(i)) for i in range(gate_number * 2)]
# create the constraints for the input
for i in range(gate_number):
    input0 = [tm.mk_term(Kind.EQUAL, [qs_v[2 * i], xs_v[j]]) for j in range(bits)]
    input1 = [tm.mk_term(Kind.EQUAL, [qs_v[2 * i + 1], xs_v[j]]) for j in range(bits)]
    inter0 = [tm.mk_term(Kind.EQUAL, [qs_v[2 * i], ts_v[j]]) for j in range(i)]
    inter1 = [tm.mk_term(Kind.EQUAL, [qs_v[2 * i + 1], ts_v[j]]) for j in range(i)]
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, input0 + inter0))
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, input1 + inter1))

In [16]:
# need encoding for gate type
gts_v = [tm.mk_const(sortbv, "gt{}".format(i)) for i in range(gate_number)]

# create the constraints for the gate type
for i in range(gate_number):
    types = [
        tm.mk_term(Kind.EQUAL, [gts_v[i], tm.mk_bv_value(sortbv, gts[j])])
        for j in range(len(gts))
    ]
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, types))

In [17]:
# create the constraints for the output, it is the hard part.
one_sort = tm.mk_bv_sort(1)
one = tm.mk_bv_value(one_sort, 1)
zores = tm.mk_bv_value(sortbv, 0)
ones = tm.mk_bv_value(sortbv, 2**bit_with - 1)
for i in range(gate_number):
    # create the constraints for the output
    gt0 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [0, 0])
    gt1 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [1, 1])
    gt2 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [2, 2])
    gt3 = tm.mk_term(Kind.BV_EXTRACT, [gts_v[i]], [3, 3])
    gt0_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt0, one]),
            zores,
            tm.mk_term(Kind.BV_AND, [qs_v[2 * i], qs_v[2 * i + 1]]),
        ],
    )
    gt1_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt1, one]),
            zores,
            qs_v[2 * i + 1],
        ],
    )
    gt2_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt2, one]),
            zores,
            qs_v[2 * i],
        ],
    )
    gt3_ul = tm.mk_term(
        Kind.ITE,
        [
            tm.mk_term(Kind.BV_ULT, [gt3, one]),
            zores,
            ones,
        ],
    )
    bitwuzla.assert_formula(
        tm.mk_term(
            Kind.EQUAL,
            [
                ts_v[i],
                tm.mk_term(
                    Kind.BV_XOR,
                    [
                        gt0_ul,
                        tm.mk_term(
                            Kind.BV_XOR,
                            [gt1_ul, tm.mk_term(Kind.BV_XOR, [gt2_ul, gt3_ul])],
                        ),
                    ],
                ),
            ],
        )
    )

In [18]:
# create the constraints for the sbox output link the gate output
for i in range(bits):
    ts_temp = [tm.mk_term(Kind.EQUAL, [ys_v[i], ts_v[j]]) for j in range(gate_number)]
    bitwuzla.assert_formula(tm.mk_term(Kind.OR, ts_temp))

In [19]:
assertions = bitwuzla.get_assertions()
print("Assertions:")
print("{")
for a in assertions:
    print(f" {a}")
print("}")

Assertions:
{
 (= x0 #b0000000011111111)
 (= x1 #b0000111100001111)
 (= x2 #b0011001100110011)
 (= x3 #b0101010101010101)
 (= y0 #b1110111011000000)
 (= y1 #b1010101100010011)
 (= y2 #b0101111100000101)
 (= y3 #b0011011101110000)
 (or (or (or (= q0 x0) (= q0 x1)) (= q0 x2)) (= q0 x3))
 (or (or (or (= q1 x0) (= q1 x1)) (= q1 x2)) (= q1 x3))
 (or (or (or (or (= q2 x0) (= q2 x1)) (= q2 x2)) (= q2 x3)) (= q2 t0))
 (or (or (or (or (= q3 x0) (= q3 x1)) (= q3 x2)) (= q3 x3)) (= q3 t0))
 (or (or (or (or (or (= q4 x0) (= q4 x1)) (= q4 x2)) (= q4 x3)) (= q4 t0)) (= q4 t1))
 (or (or (or (or (or (= q5 x0) (= q5 x1)) (= q5 x2)) (= q5 x3)) (= q5 t0)) (= q5 t1))
 (or (or (or (or (or (or (= q6 x0) (= q6 x1)) (= q6 x2)) (= q6 x3)) (= q6 t0)) (= q6 t1)) (= q6 t2))
 (or (or (or (or (or (or (= q7 x0) (= q7 x1)) (= q7 x2)) (= q7 x3)) (= q7 t0)) (= q7 t1)) (= q7 t2))
 (or (or (or (or (or (or (or (= q8 x0) (= q8 x1)) (= q8 x2)) (= q8 x3)) (= q8 t0)) (= q8 t1)) (= q8 t2)) (= q8 t3))
 (or (or (or (or (or (or (

In [20]:
smt2 = bitwuzla.print_formula()

with open("./smt2/midori.smt2", "w") as f:
    f.write(smt2)

background to run :  `nohup python3 bgc.py &`

stop `ps aux | grep "python3 bgc.py" `

`kill -9 PID`

In [ ]:
import time

start_time = time.time()
result = bitwuzla.check_sat()
end_time = time.time()

print("Time taken: ", end_time - start_time, "seconds")

In [ ]:
if result == Result.SAT:
    print("SAT")
    for x in xs_v:
        print(f"{x} = {hex(int(bitwuzla.get_value(x).value(2),2))}")
    for q in qs_v:
        print(f"{q} = {hex(int(bitwuzla.get_value(q).value(2),2))}")
    for t in ts_v:
        print(f"{t} = {hex(int(bitwuzla.get_value(t).value(2),2))}")
    for gt in gts_v:
        print(f"{gt} = {hex(int(bitwuzla.get_value(gt).value(2),2))}")
    for y in ys_v:
        print(f"{y} = {hex(int(bitwuzla.get_value(y).value(2),2))}")

SAT
x0 = 0xf
x1 = 0x33
x2 = 0x55
q0 = 0x33
q1 = 0xf
q2 = 0x33
q3 = 0x3c
q4 = 0x55
q5 = 0x33
q6 = 0x30
q7 = 0x55
q8 = 0x3c
q9 = 0x77
q10 = 0xf
q11 = 0x55
q12 = 0x3c
q13 = 0x5
t0 = 0x3c
t1 = 0x30
t2 = 0x77
t3 = 0x65
t4 = 0x4b
t5 = 0x5
t6 = 0x39
gt0 = 0x6
gt1 = 0x1
gt2 = 0x7
gt3 = 0x6
gt4 = 0x6
gt5 = 0x1
gt6 = 0x6
y0 = 0x4b
y1 = 0x39
y2 = 0x65
